In [1]:
from ultralytics import YOLO
import numpy as np
import csv
import cv2
import pandas as pd
import matplotlib.pyplot as plt


In [ ]:
import csv
import os
from deep_sort_realtime.deepsort_tracker import DeepSort
from ultralytics import YOLO
import cv2
import numpy as np

class ObjectTracker:
    def __init__(self, model_path, video_folder, pixel_to_meter):
        # Load YOLOv8 model
        self.model = YOLO(model_path)
        self.model.to('cuda')

        # Initialize DeepSORT tracker
        self.tracker = DeepSort(max_age=3, max_cosine_distance=0.01)

        # Store previous positions and tracking data
        self.previous_positions = {}
        self.tracking_data = []
        self.pixel_to_meter = pixel_to_meter  # Conversion factor from pixels to meters

        # Get all video files in the folder
        self.video_files = [f for f in os.listdir(video_folder) if f.endswith('.MOV')]
        self.video_folder = video_folder

        if not self.video_files:
            raise ValueError("No .MOV files found in the specified video folder.")

    def calculate_velocity(self, track_id, current_position, fps):
        if track_id in self.previous_positions:
            previous_position = self.previous_positions[track_id]
            displacement = np.linalg.norm(current_position - previous_position)
            velocity = displacement * fps * self.pixel_to_meter
        else:
            velocity = 0
        self.previous_positions[track_id] = current_position
        return velocity

    def draw_bounding_box(self, frame, bbox, track_id, velocity):
        cv2.rectangle(frame, (int(bbox[0]), int(bbox[1])), (int(bbox[2]), int(bbox[3])), (0, 255, 0), 2)
        cv2.putText(frame, f'ID: {track_id}', (int(bbox[0]), int(bbox[1])-20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
        cv2.putText(frame, f'Vel: {velocity:.2f} m/s', (int(bbox[0]), int(bbox[1])-40), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    def process_video(self, video_path):
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            raise Exception(f"Could not open video file: {video_path}")

        fps = cap.get(cv2.CAP_PROP_FPS)

        self.previous_positions.clear()
        self.tracking_data.clear()

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            results = self.model(frame)
            detections = results[0].boxes.data

            dets = []
            for box in detections:
                x1, y1, x2, y2, conf, cls = box.cpu().numpy()
                width = x2 - x1
                height = y2 - y1
                dets.append([[x1, y1, width, height], conf])

            tracks = self.tracker.update_tracks(dets, frame=frame)

            for track in tracks:
                if track.is_confirmed():
                    track_id = track.track_id
                    bbox = track.to_tlbr()
                    x_center = (bbox[0] + bbox[2]) / 2
                    y_center = (bbox[1] + bbox[3]) / 2
                    current_position = np.array([x_center, y_center])

                    velocity = self.calculate_velocity(track_id, current_position, fps)
                    self.draw_bounding_box(frame, bbox, track_id, velocity)

                    self.tracking_data.append([cap.get(cv2.CAP_PROP_POS_FRAMES), track_id, x_center, y_center, velocity])

        cap.release()

    def export_tracking_data(self, csv_path):
        with open(csv_path, 'w', newline='') as csvfile:
            csvwriter = csv.writer(csvfile)
            csvwriter.writerow(['Frame', 'Object ID', 'X Center', 'Y Center', 'Velocity (m/s)'])
            csvwriter.writerows(self.tracking_data)

    def run(self):
        for video_file in self.video_files:
            video_path = os.path.join(self.video_folder, video_file)

            output_folder = os.path.join(self.video_folder, video_file.replace('.MOV', '_output'))
            if not os.path.exists(output_folder):
                os.makedirs(output_folder)

            print(f"Processing video: {video_file}")
            self.process_video(video_path)
            self.export_tracking_data(os.path.join(output_folder, f'{video_file.replace(".MOV", "_tracking_data.csv")}'))

# Usage
pixel_to_meter = 0.01  # Example conversion factor
video_folder = 'data model1model2'  # Ensure this path is correct

tracker = ObjectTracker('bestYOLOV8x.pt', video_folder, pixel_to_meter)

try:
    tracker.run()
except Exception as e:
    print(f"An error occurred: {e}")


Processing video: MaMe-FeMos-FeMe-d2-25ng.MOV

0: 384x640 10 fishs, 210.0ms
Speed: 4.0ms preprocess, 210.0ms inference, 642.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 fishs, 12.0ms
Speed: 3.5ms preprocess, 12.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 fishs, 11.0ms
Speed: 2.1ms preprocess, 11.0ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 fishs, 11.0ms
Speed: 2.0ms preprocess, 11.0ms inference, 5.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 fishs, 11.6ms
Speed: 105.3ms preprocess, 11.6ms inference, 2.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 fishs, 13.5ms
Speed: 1.0ms preprocess, 13.5ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 fishs, 12.1ms
Speed: 2.1ms preprocess, 12.1ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 fishs, 82.7ms
Speed: 3.0ms preprocess, 82.7ms inference, 

In [4]:
import csv
from deep_sort_realtime.deepsort_tracker import DeepSort, Detection
from ultralytics import YOLO
import cv2
import numpy as np
import pandas as pd

class ObjectTracker:
    def __init__(self, model_path, video_path, output_path, pixel_to_meter):
        # Load YOLOv8 model
        self.model = YOLO(model_path)
        self.model.to('cuda')

        # Initialize DeepSORT tracker
        self.tracker = DeepSort(max_age=5,
                n_init=2,
                nms_max_overlap=0.9,
                max_cosine_distance=0.3,
                nn_budget=None,
                override_track_class = None,
                embedder="mobilenet",
                half=True,
                bgr=True,
                embedder_gpu=True,
                embedder_model_name=None,
                embedder_wts=None,
                polygon=False,
                today=None)

        # Load video
        self.cap = cv2.VideoCapture(video_path)
        if not self.cap.isOpened():
            raise Exception("Could not open video file.")

        # Define the codec and create VideoWriter object
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        fps = self.cap.get(cv2.CAP_PROP_FPS)
        frame_width = int(self.cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        frame_height = int(self.cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        self.out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

        # Store previous positions and tracking data
        self.previous_positions = {}
        self.tracking_data = []
        self.pixel_to_meter = pixel_to_meter  # Conversion factor from pixels to meters

    def calculate_velocity(self, track_id, current_position, fps):
        # Calculate velocity if previous position exists
        if track_id in self.previous_positions:
            previous_position = self.previous_positions[track_id]
            displacement = np.linalg.norm(current_position - previous_position)  # Euclidean distance
            velocity = displacement * fps * self.pixel_to_meter  # Convert to m/s
        else:
            velocity = 0
        self.previous_positions[track_id] = current_position
        return velocity

    def draw_bounding_box(self, frame, bbox, track_id, velocity):
        # Draw bounding box and velocity on the frame
        cv2.rectangle(frame, (int(bbox[0]), int(bbox[1])), (int(bbox[2]), int(bbox[3])), (0, 255, 0), 2)
        cv2.putText(frame, f'ID: {track_id}', (int(bbox[0]), int(bbox[1])-20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
        cv2.putText(frame, f'Vel: {velocity:.2f} m/s', (int(bbox[0]), int(bbox[1])-40), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    def process_video(self):
        fps = self.cap.get(cv2.CAP_PROP_FPS)

        while self.cap.isOpened():
            ret, frame = self.cap.read()
            if not ret:
                break

            # Perform object detection
            results = self.model(frame)
            detections = results[0].boxes.data

            # Prepare detections for DeepSORT
            dets = []
            for box in detections:
                x1, y1, x2, y2, conf, cls = box.cpu().numpy()
                width = x2 - x1
                height = y2 - y1
                dets.append([[x1, y1, width, height], conf])

            # Update tracker with detections
            tracks = self.tracker.update_tracks(dets, frame=frame)

            # Draw tracking results on the frame and log tracking data
            for track in tracks:
                if track.is_confirmed():
                    track_id = track.track_id
                    bbox = track.to_tlbr()
                    x_center = (bbox[0] + bbox[2]) / 2
                    y_center = (bbox[1] + bbox[3]) / 2
                    current_position = np.array([x_center, y_center])

                    velocity = self.calculate_velocity(track_id, current_position, fps)

                    # Draw bounding box and velocity
                    self.draw_bounding_box(frame, bbox, track_id, velocity)

                    # Append tracking data for CSV export
                    self.tracking_data.append([self.cap.get(cv2.CAP_PROP_POS_FRAMES), track_id, x_center, y_center, velocity])

            # Write the frame to the output video file
            self.out.write(frame)

            # Display the frame (optional)
            cv2.imshow('YOLOv8 with DeepSORT Tracking and Velocity', frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        # Release everything
        self.cap.release()
        self.out.release()
        cv2.destroyAllWindows()

    def export_tracking_data(self, csv_path):
        # Write tracking data to a CSV file
        with open(csv_path, 'w', newline='') as csvfile:
            csvwriter = csv.writer(csvfile)
            csvwriter.writerow(['Frame', 'Object ID', 'X Center', 'Y Center', 'Velocity (m/s)'])
            csvwriter.writerows(self.tracking_data)

# Usage
pixel_to_meter = 0.01  # Example conversion factor (1 pixel = 0.01 meters)
tracker = ObjectTracker('bestYOLOV8x.pt', '1017.mp4', 'output.mp4', pixel_to_meter)
tracker.process_video()
tracker.export_tracking_data('MOV.csv')

# Load the data for additional analysis
data = pd.read_csv('MOV.csv')

# Sort by object ID and frame number
data = data.sort_values(by=['Object ID', 'Frame'])

# Initialize an empty DataFrame for results
velocity_data = pd.DataFrame(columns=['Frame', 'Object ID', 'X Center', 'Y Center', 'Velocity', 'Part'])

# Define x-coordinate ranges for the three parts
x1 = 719.091584
x2 = 1176.909120

# Calculate velocities for each object ID
for object_id in data['Object ID'].unique():
    # Filter data for the current object
    obj_data = data[data['Object ID'] == object_id].copy()
    
    # Calculate differences in X and Y coordinates between consecutive frames
    obj_data['X_diff'] = obj_data['X Center'].diff().fillna(0)
    obj_data['Y_diff'] = obj_data['Y Center'].diff().fillna(0)
    
    # Calculate the Euclidean distance (velocity) between frames
    obj_data['Velocity'] = np.sqrt(obj_data['X_diff']**2 + obj_data['Y_diff']**2) * ((141-3)/4) * 0.01 * pixel_to_meter
    
    # Determine the part for each x-coordinate
    obj_data['Part'] = pd.cut(obj_data['X Center'], bins=[-np.inf, x1, x2, np.inf], labels=[0, 1, 2])

    # Append the results to the velocity_data DataFrame
    velocity_data = pd.concat([velocity_data, obj_data[['Frame', 'Object ID', 'X Center', 'Y Center', 'Velocity', 'Part']]])

# Group by object ID and check if velocities are different
def check_velocity_variation(group):
    velocities = group['Velocity'].values
    return len(set(velocities)) > 1

# Check each object ID for varying velocities
velocity_variation = velocity_data.groupby('Object ID').apply(check_velocity_variation)

# Filter object IDs with varying velocities
objects_with_varying_velocities = velocity_variation[velocity_variation].index

print("Object IDs with varying velocities:")
print(objects_with_varying_velocities.tolist())

# Optional: Save the filtered results to a CSV file
velocity_data.to_csv('velocity_per_frame_filtered.csv', index=False)

# Load your DataFrame (assuming you've already read the data into `df`)
df = pd.read_csv('average_velocity_with_part.csv')

# Function to calculate distance between consecutive points
def calculate_distance(group):
    group = group.sort_values('Frame')
    x_diff = group['X Center'].diff()
    y_diff = group['Y Center'].diff()
    
    # Set distance to 0 where both x_diff and y_diff are NaN
    group['Distance'] = np.where(pd.isna(x_diff) & pd.isna(y_diff), 0, np.sqrt(x_diff**2 + y_diff**2) * 0.001)
    
    return group

# Apply the function to calculate distance for each Object ID group
df = df.groupby('Object ID', group_keys=False).apply(calculate_distance)

# Calculate the total distance for each Object ID and part
total_distances = df.groupby(['Object ID', 'Part'])['Distance'].sum().reset_index()

# Rename the column for clarity
total_distances.rename(columns={'Distance': 'Total Distance (m)'}, inplace=True)

# Show the result
print(total_distances)



0: 384x640 11 fishs, 34.6ms
Speed: 4.0ms preprocess, 34.6ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 fishs, 13.1ms
Speed: 2.0ms preprocess, 13.1ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 fishs, 11.9ms
Speed: 2.0ms preprocess, 11.9ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 fishs, 13.1ms
Speed: 2.0ms preprocess, 13.1ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 fishs, 12.4ms
Speed: 2.0ms preprocess, 12.4ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 fishs, 13.2ms
Speed: 2.0ms preprocess, 13.2ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 fishs, 12.1ms
Speed: 2.0ms preprocess, 12.1ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 fishs, 12.1ms
Speed: 2.0ms preprocess, 12.1ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640

C:\Users\MAY02\AppData\Local\Temp\ipykernel_31280\4169046653.py:159: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  velocity_data = pd.concat([velocity_data, obj_data[['Frame', 'Object ID', 'X Center', 'Y Center', 'Velocity', 'Part']]])
C:\Users\MAY02\AppData\Local\Temp\ipykernel_31280\4169046653.py:159: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  velocity_data = pd.concat([velocity_data, obj_data[['Frame', 'Object ID', 'X Center', 'Y Center', 'Velocity', 'Part']]])
C:\Users\MAY02\AppData\Local

Object IDs with varying velocities:
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 16, 18, 20, 21, 22, 24, 28, 29, 30, 32, 33, 34, 35, 36, 37, 39, 40, 41, 42, 43, 44, 45, 46, 50, 51, 52, 53, 55, 57, 58, 60, 62, 65, 66, 67, 68, 70, 72, 74, 75, 76, 77, 78, 79, 81, 83, 84, 85, 87, 88, 89, 90, 91, 92, 93, 94, 95, 98, 99, 101, 102, 103, 104, 105, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 118, 120, 121, 123, 124, 125, 126, 127, 130, 131]


C:\Users\MAY02\AppData\Local\Temp\ipykernel_31280\4169046653.py:193: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('Object ID', group_keys=False).apply(calculate_distance)


KeyError: 'Part'